# Manuscript code

In this notebook, the necessary code to generate the figures in the publication is provided. 

Models are provided as Pytorch files with weights only. The architecture are specified in this notebook, so that these files may be read correctly.

## Library imports and general parameters/variables

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import joblib
import oyaml as yaml
import pickle

import pandas as pd
import numpy as np
import cantera as ct

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme("notebook")

from chem_ai.cantera_runs import compute_nn_cantera_0D_homo
from chem_ai.utils import get_molar_mass_atomic_matrix

In [ ]:
torch.set_default_dtype(torch.float64)

We create a dictionary with data for each case. The entries of the dictionary are dictionaries including all the parameters needed, the folder location, and will later include the NN models as lists. The parameters of the cases are the following:

+ *fuel*: fuel used for the simulation
+ *nbdt*: number of time steps used for the multi-database generation
+ *dt*: time step used for the trajectory discretisation in time
+ *extend*: extension factor of time steps (if >1 training is made for a time step interval larger than the targeted values)
+ *nsamp*: number of 0D trajectories used for training/validation/testing.
+ *nn_type*: type of NN architecture used (baseline or deeponet). 

In [ ]:
data = {
    "case_1": {"fuel": "H2", "nbdt": 1, "dt": 0.5e-6, "extend": 1.0, "nsamp": 200, "nn_type": "baseline", "folder": "case_0D_multidt_H2_nbdt1_dt0.5m6_extend1.0_nsamp200", "models_list": [], "test_results": [], "fitness": [], "fitness_stats": []},
    "case_2": {"fuel": "H2", "nbdt": 1, "dt": 1e-6, "extend": 1.0, "nsamp": 200, "nn_type": "baseline", "folder": "case_0D_multidt_H2_nbdt1_dt1m6_extend1.0_nsamp200", "models_list": [], "test_results": [], "fitness": [], "fitness_stats": []},
    "case_3": {"fuel": "H2", "nbdt": 1, "dt": 1e-6, "extend": 1.0, "nsamp": 400, "nn_type": "baseline", "folder": "case_0D_multidt_H2_nbdt1_dt1m6_extend1.0_nsamp400", "models_list": [], "test_results": [], "fitness": [], "fitness_stats": []},
    "case_4": {"fuel": "H2", "nbdt": 1, "dt": 1e-6, "extend": 1.5, "nsamp": 200, "nn_type": "baseline", "folder": "case_0D_multidt_H2_nbdt1_dt1m6_extend1.5_nsamp200", "models_list": [], "test_results": [], "fitness": [], "fitness_stats": []},
    "case_5": {"fuel": "H2", "nbdt": 2, "dt": 1e-6, "extend": 1.0, "nsamp": 200, "nn_type": "baseline", "folder": "case_0D_multidt_H2_nbdt2_dt1m6_extend1.0_nsamp200", "models_list": [], "test_results": [], "fitness": [], "fitness_stats": []},
    "case_6": {"fuel": "H2", "nbdt": 4, "dt": 1e-6, "extend": 1.0, "nsamp": 200, "nn_type": "baseline", "folder": "case_0D_multidt_H2_nbdt4_dt1m6_extend1.0_nsamp200", "models_list": [], "test_results": [], "fitness": [], "fitness_stats": []},
    "case_7": {"fuel": "NH3", "nbdt": 1, "dt": 1e-6, "extend": 1.0, "nsamp": 200, "nn_type": "baseline", "folder": "case_0D_multidt_NH3_nbdt1_dt1m6_extend1.0_nsamp200", "models_list": [], "test_results": [], "fitness": [], "fitness_stats": []},
    "case_8": {"fuel": "NH3", "nbdt": 1, "dt": 1e-6, "extend": 1.0, "nsamp": 400, "nn_type": "baseline", "folder": "case_0D_multidt_NH3_nbdt1_dt1m6_extend1.0_nsamp400", "models_list": [], "test_results": [], "fitness": [], "fitness_stats": []},
    "case_9": {"fuel": "H2", "nbdt": 1, "dt": 1e-6, "extend": 1.0, "nsamp": 200, "nn_type": "deeponet", "folder": "case_0D_multidt_H2_nbdt1_dt1m6_extend1.0_nsamp200", "models_list": [], "test_results": [], "fitness": [], "fitness_stats": []},
}

## Loading the Pytorch models

Here are the architectures of the models used in this work. These architectures are needed to read the network weights. Reading weights only enables a better compatibility between platforms. 

In [ ]:
act_func = nn.GELU

n_in_H2 = 10
n_in_NH3 = 30
n_out_H2 = 9
n_out_NH3 = 29

In [ ]:
class ChemNN_multi_H2(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(n_in_H2 + 1, 100)
        self.act1 = act_func()
        self.hidden2 = nn.Linear(100, 50)
        self.act2 = act_func()
        self.hidden3 = nn.Linear(50, 50)
        self.act3 = act_func()
        self.output = nn.Linear(50, n_out_H2)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        x = self.output(x)
        return x

In [ ]:
class ChemNN_multi_NH3(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(n_in_NH3 + 1, 150)
        self.act1 = act_func()
        self.hidden2 = nn.Linear(150, 100)
        self.act2 = act_func()
        self.hidden3 = nn.Linear(100, 100)
        self.act3 = act_func()
        self.output = nn.Linear(100, n_out_NH3)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        x = self.output(x)
        return x


In [ ]:
n_neurons = 10

class DeepONet_shift_H2(nn.Module):
    def __init__(self):
        super().__init__()

        # Branch net
        self.b_hidden1 = nn.Linear(n_in_H2, 40)
        self.b_act1 = act_func()
        self.b_hidden2 = nn.Linear(40, 40)
        self.b_act2 = act_func()
        self.b_hidden3 = nn.Linear(40, 40)
        self.b_act3 = act_func()
        self.b_output = nn.Linear(40, n_neurons*n_out_H2)

        # Trunk net
        self.t_hidden1 = nn.Linear(1, 20)
        self.t_act1 = act_func()
        self.t_hidden2 = nn.Linear(20, 20)
        self.t_act2 = act_func()
        # self.t_hidden3 = nn.Linear(20, 20)
        # self.t_act3 = act_func()
        self.t_output = nn.Linear(20, n_neurons*n_out_H2)

        # Shift net
        self.s_hidden1 = nn.Linear(n_in_H2, 10)
        self.s_act1 = act_func()
        self.s_hidden2 = nn.Linear(10, 10)
        self.s_act2 = act_func()
        self.s_output = nn.Linear(10, 1)
        # self.s_act3 = act_func()

 
    def forward(self, x):

        dt = x[:,-1]
        y = x[:,:-1]

        dt = dt.reshape((x.shape[0], 1))

        s = self.s_act1(self.s_hidden1(y))
        s = self.s_act2(self.s_hidden2(s))
        # s = self.s_act3(self.s_output(s))
        s = self.s_output(s)

        b = self.b_act1(self.b_hidden1(y))
        b = self.b_act2(self.b_hidden2(b))
        b = self.b_act3(self.b_hidden3(b))
        b = self.b_output(b)

        # dt_s = dt + torch.log(s+1.0e-10)
        dt_s = dt + s

        t = self.t_act1(self.t_hidden1(dt_s))
        t = self.t_act2(self.t_hidden2(t))
        # t = self.t_act3(self.t_hidden3(t))
        t = self.t_output(t)

        y_dt = torch.zeros((x.shape[0],n_out_H2))

        # Reshape b and t for batched dot product
        b = b.reshape((x.shape[0], n_out_H2, n_neurons))  # (batch_size, n_out, 10)
        t = t.reshape((x.shape[0], n_out_H2, n_neurons))  # (batch_size, n_out, 10)

        # Perform batched dot product along the last dimension
        y_dt = torch.sum(b * t, dim=2)  # (batch_size, n_out)

        return y_dt

In [ ]:
for key in data:

    folder = data[key]["folder"]

    if data[key]["nn_type"]=="baseline":

        if data[key]["fuel"]=="H2":

            for i in range(1,6):
                model = ChemNN_multi_H2()
                model_file = os.path.join(folder,f"nn_model_mlp_{i}/pytorch_mlp.pt")
                model.load_state_dict(torch.load(model_file, weights_only=True))
                data[key]["models_list"].append(model)

        elif data[key]["fuel"]=="NH3":

            for i in range(1,6):
                model = ChemNN_multi_NH3()
                model_file = os.path.join(folder,f"nn_model_mlp_{i}/pytorch_mlp.pt")
                model.load_state_dict(torch.load(model_file, weights_only=True))
                data[key]["models_list"].append(model)

    if data[key]["nn_type"]=="deeponet": # deeponet only done for H2 so far, to change later if necessary

        for i in range(1,6):
            model = DeepONet_shift_H2()
            model_file = os.path.join(folder,f"nn_model_deeponet_{i}/pytorch_deeponet.pt")
            model.load_state_dict(torch.load(model_file, weights_only=True))
            data[key]["models_list"].append(model)


## Function to compute results and metrics on the test database

In [ ]:
def run_test_simulations(dt, folder, dtb_params, df_sim_test, model, Xscaler, Yscaler, Tscaler):

    fuel = dtb_params["fuel"]
    mech_file = dtb_params["mech_file"]
    gas = ct.Solution(mech_file)
    A_element = get_molar_mass_atomic_matrix(gas.species_names, fuel, True)

    # --------------- RUNNING TEST SIMULATIONS -------------------------

    df_sim_test = pd.read_csv(os.path.join(folder, "sim_test.csv"))

    n_sim = df_sim_test.shape[0]
    print(f"There are {n_sim} test simulations")

    list_test_results = []

    fails = 0
    for i, row in df_sim_test.iterrows():

        phi_ini = row['Phi']
        temperature_ini = row['T0']

        print(f"Performing test computation for phi={phi_ini}; T0={temperature_ini}")

        device = torch.device('cpu')
        df_exact, df_nn, fail = compute_nn_cantera_0D_homo(device, model, Xscaler, Yscaler, phi_ini, temperature_ini, dt, dtb_params, A_element, 1, Tscaler, False)

        fails += fail

        list_test_results.append((df_exact, df_nn))


    print(f"dt={dt}:Total number of simulations which crashed: {fails}")


    return list_test_results

In [ ]:
def compute_fitness(list_test_results, dtb_params, df_sim_test, Xscaler):

    n_sim = df_sim_test.shape[0]

    # --------------- COMPUTING FITNESS -------------------------
    fuel = dtb_params["fuel"]
    log_transform = dtb_params["log_transform"]
    threshold = dtb_params["threshold"]

    if fuel=="H2":
        n_out = 9
    elif fuel=="NH3":
        n_out = 29

    # Results will be stored in data_errors array.
    # The first column corresponds to errors on temperature
    # The next n_out columns correspond to errors on species mass fractions
    # The last column corresponds to the mean error
    data_errors = np.empty([n_sim, n_out+2]) 

    for i_sim in range(n_sim):

        df_exact = list_test_results[i_sim][0]
        df_nn = list_test_results[i_sim][1]

        # Removing undesired variables
        df_exact = df_exact.drop('Time', axis=1)
        df_nn = df_nn.drop(["Time","SumYk", "Y_C", "Y_H", "Y_O", "Y_N"], axis=1)

        # Applying log
        if log_transform:

            df_exact[df_exact < threshold] = threshold
            df_nn[df_nn < threshold] = threshold

            df_exact.iloc[:, 1:] = np.log(df_exact.iloc[:, 1:])
            df_nn.iloc[:, 1:] = np.log(df_nn.iloc[:, 1:])

        # Scaling
        data_exact_scaled = (df_exact-Xscaler.mean)/(Xscaler.std+1.0e-7)
        data_nn_scaled = (df_nn-Xscaler.mean)/(Xscaler.std+1.0e-7)

        diff_exact_nn = np.abs(data_nn_scaled-data_exact_scaled)

        diff_exact_nn = diff_exact_nn.mean(axis=0)

        M = diff_exact_nn.mean()

        print(f"Simulation {i_sim} error M = {M}")

        data_errors[i_sim, :n_out+1] = diff_exact_nn
        data_errors[i_sim, n_out+1] = M


    return data_errors

## Running test simulations

In [ ]:
for i_model in range(5):
    data[key]["test_results"].append({})
    data[key]["fitness"].append({})
    data[key]["fitness_stats"].append({})

In [ ]:
# Time steps which are considered
dt_list = [0.1e-5, 0.2e-5, 0.3e-5, 0.4e-5, 0.5e-5, 0.6e-5, 0.7e-5, 0.8e-5, 0.9e-5, 1e-5]

for key in data:
    
    folder = data[key]["folder"]
    print(f" ------- RUNNING SIMULATIONS FOR CASE {folder} ------- \n")

    with open(os.path.join(folder, "dtb_params.yaml"), "r") as file:
        dtb_params = yaml.safe_load(file)

    df_sim_test = pd.read_csv(os.path.join(folder, "sim_test.csv"))

    Xscaler = joblib.load(os.path.join(folder, "processed_database", "Xscaler.pkl"))
    Yscaler = joblib.load(os.path.join(folder, "processed_database", "Yscaler.pkl"))
    Tscaler = joblib.load(os.path.join(folder, "processed_database", "Tscaler.pkl"))

    # Loop on the 5 models
    for i_model in range(5):
        print(f"   >>>> Running with model {i_model}")

        model = data[key]["models_list"][i_model]

        data[key]["test_results"].append({})
        data[key]["fitness"].append({})
        data[key]["fitness_stats"].append({})

        for dt in dt_list:

            print(f"DT={dt}")

            list_test_results = run_test_simulations(dt, folder, dtb_params, df_sim_test, model, Xscaler, Yscaler, Tscaler)
            data[key]["test_results"][i_model][dt] = list_test_results


            data_errors = compute_fitness(list_test_results, dtb_params, df_sim_test, Xscaler)
            data[key]["fitness"][i_model][dt] = data_errors

        
        data_errors_mean = np.empty(len(dt_list))
        data_errors_std = np.empty(len(dt_list))

        for i, dt in enumerate(dt_list):
            data_errors_mean[i] = data[key]["fitness"][i_model][dt][:,-1].mean()
            data_errors_std[i] = data[key]["fitness"][i_model][dt][:,-1].std()

        data[key]["fitness_stats"][i_model] = (data_errors_mean, data_errors_std)


In [ ]:
with open("article_data.pkl", "wb") as file:
    pickle.dump(data, file)

In [ ]:
# with open("article_data.pkl", "rb") as file:
#     loaded_data = pickle.load(file)

## Plotting results

### Comparing Vanilla and DeepOnet strategies

In [ ]:
vanilla_case = "case_2"
deeponet_case = "case_9"

# Matrices with rows representing models and columns each dt value
fitness_mean_vanilla = np.empty((5, len(dt_list)))
fitness_mean_deeponet = np.empty((5, len(dt_list)))

for i_model in range(5):
    fitness_mean_vanilla[i_model,:] = data[vanilla_case]["fitness_stats"][i_model][0]
    fitness_mean_deeponet[i_model,:] = data[deeponet_case]["fitness_stats"][i_model][0]


fitness_mean_vanilla_avg = fitness_mean_vanilla.mean(axis=0)
fitness_mean_vanilla_std = fitness_mean_vanilla.std(axis=0)
#
fitness_mean_deeponet_avg = fitness_mean_deeponet.mean(axis=0)
fitness_mean_deeponet_std = fitness_mean_deeponet.std(axis=0)

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(dt_list, fitness_mean_vanilla_avg, yerr=fitness_mean_vanilla_std, fmt='o', capsize=4, capthick=2, color='k', ecolor='k', linestyle='--', label="Vanilla NN")
ax.errorbar(dt_list, fitness_mean_deeponet_avg, yerr=fitness_mean_deeponet_std, fmt='^', capsize=4, capthick=2, color='r', ecolor='r', linestyle='--', label="DeepONet")

ax.set_xlabel("$dt$ $[s]$", fontsize=14)
ax.set_ylabel("Error $[-]$", fontsize=14)

ax.set_ylim([0.0, 0.07])

ax.legend()

fig.tight_layout()

### Influence of time step sampling

In [ ]:
nbdt1 = "case_2"
nbdt2 = "case_5"
nbdt4 = "case_6"

# Matrices with rows representing models and columns each dt value
fitness_mean_nbdt1 = np.empty((5, len(dt_list)))
fitness_mean_nbdt2 = np.empty((5, len(dt_list)))
fitness_mean_nbdt4 = np.empty((5, len(dt_list)))

for i_model in range(5):
    fitness_mean_nbdt1[i_model,:] = data[nbdt1]["fitness_stats"][i_model][0]
    fitness_mean_nbdt2[i_model,:] = data[nbdt2]["fitness_stats"][i_model][0]
    fitness_mean_nbdt4[i_model,:] = data[nbdt4]["fitness_stats"][i_model][0]


fitness_mean_nbdt1_avg = fitness_mean_nbdt1.mean(axis=0)
fitness_mean_nbdt1_std = fitness_mean_nbdt1.std(axis=0)
#
fitness_mean_nbdt2_avg = fitness_mean_nbdt2.mean(axis=0)
fitness_mean_nbdt2_std = fitness_mean_nbdt2.std(axis=0)
#
fitness_mean_nbdt4_avg = fitness_mean_nbdt4.mean(axis=0)
fitness_mean_nbdt4_std = fitness_mean_nbdt4.std(axis=0)

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(dt_list, fitness_mean_nbdt1_avg, yerr=fitness_mean_nbdt1_std, fmt='o', capsize=4, capthick=2, color='k', ecolor='k', linestyle='--', label="$n_{dt}=1$")
ax.errorbar(dt_list, fitness_mean_nbdt2_avg, yerr=fitness_mean_nbdt2_std, fmt='^', capsize=4, capthick=2, color='r', ecolor='r', linestyle='--', label="$n_{dt}=2$")
ax.errorbar(dt_list, fitness_mean_nbdt4_avg, yerr=fitness_mean_nbdt4_std, fmt='s', capsize=4, capthick=2, color='b', ecolor='b', linestyle='--', label="$n_{dt}=4$")

ax.set_xlabel("$dt$ $[s]$", fontsize=14)
ax.set_ylabel("Error $[-]$", fontsize=14)

ax.set_ylim([0.0, 0.07])

ax.legend()

fig.tight_layout()

### Comparing strategies to improve predictions

In [ ]:
baseline = "case_2"
nsamp400 = "case_3"
small_dt = "case_1"
extend_dt = "case_4"

# Matrices with rows representing models and columns each dt value
fitness_mean_baseline = np.empty((5, len(dt_list)))
fitness_mean_nsamp400 = np.empty((5, len(dt_list)))
fitness_mean_small_dt = np.empty((5, len(dt_list)))
fitness_mean_extend_dt = np.empty((5, len(dt_list)))

for i_model in range(5):
    fitness_mean_baseline[i_model,:] = data[baseline]["fitness_stats"][i_model][0]
    fitness_mean_nsamp400[i_model,:] = data[nsamp400]["fitness_stats"][i_model][0]
    fitness_mean_small_dt[i_model,:] = data[small_dt]["fitness_stats"][i_model][0]
    fitness_mean_extend_dt[i_model,:] = data[extend_dt]["fitness_stats"][i_model][0]


fitness_mean_baseline_avg = fitness_mean_baseline.mean(axis=0)
fitness_mean_baseline_std = fitness_mean_baseline.std(axis=0)
#
fitness_mean_nsamp400_avg = fitness_mean_nsamp400.mean(axis=0)
fitness_mean_nsamp400_std = fitness_mean_nsamp400.std(axis=0)
#
fitness_mean_small_dt_avg = fitness_mean_small_dt.mean(axis=0)
fitness_mean_small_dt_std = fitness_mean_small_dt.std(axis=0)
#
fitness_mean_extend_dt_avg = fitness_mean_extend_dt.mean(axis=0)
fitness_mean_extend_dt_std = fitness_mean_extend_dt.std(axis=0)

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(dt_list, fitness_mean_baseline_avg, yerr=fitness_mean_baseline_std, fmt='o', capsize=4, capthick=2, color='k', ecolor='k', linestyle='--', label="Baseline")
ax.errorbar(dt_list, fitness_mean_nsamp400_avg, yerr=fitness_mean_nsamp400_std, fmt='^', capsize=4, capthick=2, color='r', ecolor='r', linestyle='--', label="$n_{samp}=400$")
ax.errorbar(dt_list, fitness_mean_small_dt_avg, yerr=fitness_mean_small_dt_std, fmt='s', capsize=4, capthick=2, color='b', ecolor='b', linestyle='--', label="$dt_{samp}=0.5 \cdot 10^{-6}$")
ax.errorbar(dt_list, fitness_mean_extend_dt_avg, yerr=fitness_mean_extend_dt_std, fmt='s', capsize=4, capthick=2, color='g', ecolor='g', linestyle='--', label="extend")

ax.set_xlabel("$dt$ $[s]$", fontsize=14)
ax.set_ylabel("Error $[-]$", fontsize=14)

ax.set_ylim([0.0, 0.07])

ax.legend()

fig.tight_layout()

### $NH_3$ testing

In [ ]:
nh3_samp200 = "case_7"
nh3_samp400 = "case_8"

# Matrices with rows representing models and columns each dt value
fitness_mean_nh3_samp200 = np.empty((5, len(dt_list)))
fitness_mean_nh3_samp400 = np.empty((5, len(dt_list)))

for i_model in range(5):
    fitness_mean_nh3_samp200[i_model,:] = data[nh3_samp200]["fitness_stats"][i_model][0]
    fitness_mean_nh3_samp400[i_model,:] = data[nh3_samp400]["fitness_stats"][i_model][0]


fitness_mean_nh3_samp200_avg = fitness_mean_nh3_samp200.mean(axis=0)
fitness_mean_nh3_samp200_std = fitness_mean_nh3_samp200.std(axis=0)
#
fitness_mean_nh3_samp400_avg = fitness_mean_nh3_samp400.mean(axis=0)
fitness_mean_nh3_samp400_std = fitness_mean_nh3_samp400.std(axis=0)

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(dt_list, fitness_mean_nh3_samp200_avg, yerr=fitness_mean_nh3_samp200_std, fmt='o', capsize=4, capthick=2, color='k', ecolor='k', linestyle='--', label="$n_{samp}=200$")
ax.errorbar(dt_list, fitness_mean_nh3_samp400_avg, yerr=fitness_mean_nh3_samp400_std, fmt='^', capsize=4, capthick=2, color='r', ecolor='r', linestyle='--', label="$n_{samp}=400$")

ax.set_xlabel("$dt$ $[s]$", fontsize=14)
ax.set_ylabel("Error $[-]$", fontsize=14)

ax.set_ylim([0.0, 0.07])

ax.legend()

fig.tight_layout()